<a href="https://colab.research.google.com/github/batucul/calisma/blob/main/Sikayetvar_grabber_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas openpyxl gradio pyngrok==4.1.1 lxml requests beautifulsoup4

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import lxml
import re
import time
import random

def setup_session():
    """Requests oturumunu yapılandırır ve döndürür."""
    session = requests.Session()
    retry = Retry(connect=5, backoff_factor=1)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def get_page_count(session, url):
    """Belirtilen URL için sayfa sayısını alır."""
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
    site_request = session.get(url, headers=headers).text
    soup = BeautifulSoup(site_request, "lxml")
    return int(soup.find_all("a", class_="page")[-1].text)

def get_complaint_links(session, url, page_count):
    """Tüm şikayet linklerini toplar ve döndürür."""
    yorum_linkleri = []
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
    for page in range(1, page_count + 1):
        page_url = f"{url}?page={page}"
        page_request = session.get(page_url, headers=headers).text
        soup = BeautifulSoup(page_request, "lxml")
        for container in soup.find_all("div", class_="read-more-container"):
            read_more_div = container.find("div", class_="read-more")
            if read_more_div and read_more_div.has_attr('data-url'):
                data_url = read_more_div['data-url']
                yorum_linkleri.append("https://www.sikayetvar.com" + data_url)
    return yorum_linkleri

def scrape_complaint(session, url):
    """Tek bir şikayet sayfasını kazır."""
    review_site_request = session.get(url, headers=headers).text
    soup = BeautifulSoup(review_site_request, "lxml")
    try:
        baslik = soup.find("h1", class_="complaint-detail-title").text.strip()
        yorum = soup.find("div", class_="complaint-detail-description").text.strip()
        tarih = soup.find("div", class_="post-time").get_text().split("  ")[0]
        goruntulenme = soup.find("span", class_="js-view-count").get_text()
        kullanici_adi = soup.find("span", class_="username").text.strip()
        return [url, kullanici_adi, tarih, baslik, yorum, goruntulenme]
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return None

def main():
    kargo = ['tazedirekt']
    df_columns = ['PrimaryKey', 'href', 'kullanici_adi', 'tarih', 'baslik', 'yorum', 'goruntulenme']
    data = []

    session = setup_session()
    for firma in kargo:
        url = f"https://www.sikayetvar.com/{firma}"
        page_count = get_page_count(session, url)
        yorum_linkleri = get_complaint_links(session, url, page_count)

        for idx, link in enumerate(yorum_linkleri, 1):
            time.sleep(random.uniform(1, 2))
            complaint_data = scrape_complaint(session, link)
            if complaint_data:
                data.append([idx] + complaint_data)
                print(f"Scraping progress: {idx}/{len(yorum_linkleri)}")

    df = pd.DataFrame(data, columns=df_columns)
    excel_filename = "scraped_data.xlsx"
    df.to_excel(excel_filename, index=False)
    print("Scraping completed and data written to 'scraped_data.xlsx'")

if __name__ == "__main__":
    main()


In [ ]:
#%%writefile streamlit_app.py
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import lxml
import re
import time
import random
import gradio as gr
from tqdm import tqdm


df_columns = ['PrimaryKey', 'href', 'kullanici_adi', 'tarih', 'baslik', 'yorum', 'goruntulenme']


def setup_session():
    """Requests oturumunu yapılandırır ve döndürür."""
    session = requests.Session()
    retry = Retry(connect=5, backoff_factor=1)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def get_page_count(session, url):
    """Belirtilen URL için sayfa sayısını alır."""
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
    site_request = session.get(url, headers=headers).text
    soup = BeautifulSoup(site_request, "lxml")
    return int(soup.find_all("a", class_="page")[-1].text)

def get_complaint_links(session, url, page_count):
    """Tüm şikayet linklerini toplar ve döndürür."""
    yorum_linkleri = []
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
    for page in range(1, page_count + 1):
        page_url = f"{url}?page={page}"
        page_request = session.get(page_url, headers=headers).text
        soup = BeautifulSoup(page_request, "lxml")
        for container in soup.find_all("div", class_="read-more-container"):
            read_more_div = container.find("div", class_="read-more")
            if read_more_div and read_more_div.has_attr('data-url'):
                data_url = read_more_div['data-url']
                yorum_linkleri.append("https://www.sikayetvar.com" + data_url)
    return yorum_linkleri

def scrape_complaint(session, url):
    """Tek bir şikayet sayfasını kazır."""
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
    review_site_request = session.get(url, headers=headers).text
    soup = BeautifulSoup(review_site_request, "lxml")
    try:
        baslik = soup.find("h1", class_="complaint-detail-title").text.strip()
        yorum = soup.find("div", class_="complaint-detail-description").text.strip()
        tarih = soup.find("div", class_="post-time").get_text().split("  ")[0]
        goruntulenme = soup.find("span", class_="js-view-count").get_text()
        kullanici_adi = soup.find("span", class_="username").text.strip()
        return [url, kullanici_adi, tarih, baslik, yorum, goruntulenme]
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return None

def scrape_and_save(kargo,  progress=gr.Progress()):
    session = setup_session()
    url = f"https://www.sikayetvar.com/{kargo}"
    page_count = get_page_count(session, url)
    yorum_linkleri = get_complaint_links(session, url, page_count)

    data = []
    for idx, link in enumerate(yorum_linkleri):
        time.sleep(random.uniform(0, 1))
        complaint_data = scrape_complaint(session, link)
        if complaint_data:
            data.append([idx] + complaint_data)
            print(f"Scraping progress: {idx}/{len(yorum_linkleri)}")
        progress((idx + 1) / len(yorum_linkleri))

    df = pd.DataFrame(data, columns=df_columns)
    excel_filename = f"{kargo}_scraped_data.xlsx"
    df.to_excel(excel_filename, index=False)
    return excel_filename
# Create a Gradio interface
iface = gr.Interface(
    fn=scrape_and_save,
    inputs=gr.Textbox(label="Firma"),
    outputs=gr.File(),
    title="Şikayetvar Verilerini Çekme",
    description= """1- Şikayetlerini indirmek istediğiniz firmayı şikayetvar.com'da bulun
2- Firmayı bulduktan sonra adres çubuğundaki firma ismini aşağıya girin
3- Örneğin: https://www.sikayetvar.com/aras-kargo ise programa aras-kargo yazın"""
).queue().launch(share=True, debug=True, show_error=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ab953a112f13f18c12.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Scraping progress: 0/3163
Scraping progress: 1/3163
Scraping progress: 2/3163
Scraping progress: 3/3163
Scraping progress: 4/3163
Scraping progress: 5/3163
Scraping progress: 6/3163
Scraping progress: 7/3163
Scraping progress: 8/3163
Scraping progress: 9/3163
Scraping progress: 10/3163
Scraping progress: 11/3163
Scraping progress: 12/3163
Scraping progress: 13/3163
Scraping progress: 14/3163
Scraping progress: 15/3163
Scraping progress: 16/3163
Scraping progress: 17/3163
Scraping progress: 18/3163
Scraping progress: 19/3163
Scraping progress: 20/3163
Scraping progress: 21/3163
Scraping progress: 22/3163
Scraping progress: 23/3163
Scraping progress: 24/3163
Scraping progress: 25/3163
Scraping progress: 26/3163
Scraping progress: 27/3163
Scraping progress: 28/3163
Scraping progress: 29/3163
Scraping progress: 30/3163
Scraping progress: 31/3163
Scraping progress: 32/3163
Scraping progress: 33/3163
Scraping progress: 34/3163
Scraping progress: 35/3163
Scraping progress: 36/3163
Scraping pr

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-4' coro=<Queue.start_progress_updates() done, defined at /usr/local/lib/python3.10/dist-packages/gradio/queueing.py:273> exception=KeyError('26upsydow79')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 291, in start_progress_updates
    self.send_message(
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 123, in send_message
    messages = self.pending_messages_per_session[event.session_hash]
KeyError: '26upsydow79'


Scraping progress: 2736/3163
Scraping progress: 2737/3163
Scraping progress: 2738/3163


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/exceptions.py", line 68, in __call__
    await self.app(scope, receive, sender)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/middleware/asyncexitstack.py", line 20, in __call__
    raise e
  File "/usr/local/lib/python3.10/dist-packages/fastapi/middleware/asyncexitstack.py", line 17, in __call__
    await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/routing.py", line 718, in __call__
    await route.handle(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/routing.py", line 276, in handle
    await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/routing.py", line 69, in app
    await response(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/responses.py", line 270, in __call__
    async with

Scraping progress: 2739/3163
Scraping progress: 2740/3163
Scraping progress: 2741/3163
Scraping progress: 2742/3163
Scraping progress: 2743/3163
Scraping progress: 2744/3163
Scraping progress: 2745/3163
Scraping progress: 2746/3163
Scraping progress: 2747/3163
Scraping progress: 2748/3163
Scraping progress: 2749/3163
Scraping progress: 2750/3163
Scraping progress: 2751/3163
Scraping progress: 2752/3163
Scraping progress: 2753/3163
Scraping progress: 2754/3163
Scraping progress: 2755/3163
Scraping progress: 2756/3163
Scraping progress: 2757/3163
Scraping progress: 2758/3163
Scraping progress: 2759/3163
Scraping progress: 2760/3163
Scraping progress: 2761/3163
Scraping progress: 2762/3163
Scraping progress: 2763/3163
Scraping progress: 2764/3163
Scraping progress: 2765/3163
Scraping progress: 2766/3163
Scraping progress: 2767/3163
Scraping progress: 2768/3163
Scraping progress: 2769/3163
Scraping progress: 2770/3163
Scraping progress: 2771/3163
Scraping progress: 2772/3163
Scraping progr

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 561, in process_events
    self.send_message(
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 123, in send_message
    messages = self.pending_messages_per_session[event.session_hash]
KeyError: '26upsydow79'


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ab953a112f13f18c12.gradio.live


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import lxml
import re
import time
import random

In [ ]:
def setup_session():
    """Requests oturumunu yapılandırır ve döndürür."""
    session = requests.Session()
    retry = Retry(connect=5, backoff_factor=1)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [ ]:
def scrape_complaint(session, url):
    """Tek bir şikayet sayfasını kazır."""
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
    review_site_request = session.get(url, headers=headers).text
    soup = BeautifulSoup(review_site_request, "lxml")
    try:
        baslik = soup.find("h1", class_="complaint-detail-title").text.strip()
        yorum = soup.find("div", class_="complaint-detail-description").text.strip()
        tarih = soup.find("div", class_="post-time").get_text().split("  ")[0]
        goruntulenme = soup.find("span", class_="js-view-count").get_text()
        kullanici_adi = soup.find("span", class_="username").text.strip()
        return [url, kullanici_adi, tarih, baslik, yorum, goruntulenme]
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return None

In [ ]:
session=setup_session()

In [ ]:
scrape_complaint(session ,'https://www.sikayetvar.com/ethica-sigorta/ethica-sigorta-trafik-sigortasi-hasar-odemesi-1')

['https://www.sikayetvar.com/ethica-sigorta/ethica-sigorta-trafik-sigortasi-hasar-odemesi-1',
 'Kaan',
 ' 19 Ocak',
 'Ethica Sigorta Trafik Sigortası Hasar Ödemesi',
 '07.10.2021 tarihinde Ethica sigortalı aracım ile küçük çapta bir maddi hasarlı bir kaza yaşadım.Sigorta ve kaskosu tam olan kendi aracımın ve karşı aracın tüm onarım işlemleri yine sigorta ve kasko üzerinden sorunsuz şekilde tamamlanmıştı. Ocak 2022 itibari ile karşı tarafın sigorta şirketi avukatları eksik tutar gerekçesiyle tarafıma rücu etti.Sorunumu Ethica\'ya dile getirdiğimde santral görevlileri sürekli beni birilerine iletmekte ama bu birileri asla sorunu çözecek kapasitede ve yardımcı olma anlayışında insanlar olmamaktadır.Ethica tarafıma ilgili dosya no için önce ödeme yapıldığını, sonradan ise eksik yapılmış olabileceğini, söyleyip "bilemeyiz" gibi yarım ağız bir cevapla tarafıma bildirerek beni master destek firmasına yönlendirdi, orası da bana aynı süreci baştan yaşattı. Sadece gereksiz trafik ve laf kalabalı